In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt  # plt is easier to show images in notebook
import imutils

template_placeholder = "lotus.png"
template = cv2.imread("templates/lotus.png")

image_list_placeholder = []

template_gray = cv2.cvtColor(template, cv2.COLOR_BGRA2GRAY)
(tH, tW) = template_gray.shape[:2]


found = None

template_results = []
p = []
for im in image_list_placeholder:
    gray_page = cv2.cvtColor(im, cv2.COLOR_BGRA2GRAY)

    p.append(gray_page)
    for scale in np.linspace(0.2, 1.0, 20)[::-1]:  # start from 100% of the original size of the image and work our way down to 20% of the original size in 20 equally sized percent chunks
        resized_im = imutils.resize(gray_page, width = int(gray_page.shape[1] * scale))
        resizing_ratio = gray_page.shape[1] / float(resized_im.shape[1])  # ratio of the resizing

        if resized_im.shape[0] < tH or resized_im.shape[1] < tW:  # break if smaller than template
            break

        result = cv2.matchTemplate(resized_im, template_gray, cv2.TM_CCOEFF_NORMED)
        # another result method with edges below. Rosenbrock uses with text.
        # edged = cv2.Canny(resized, 50, 200)
		# result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)

        # max val is what I need to use as a threshold to see if correct id
        (minVal, maxVal, minLoc, maxLoc)  = cv2.minMaxLoc(result)

        if found is None or maxVal > found[0]:
            found = (maxVal, maxLoc, resizing_ratio)

    template_results.append(found)
    found = None

# math to convert the sizing match correctly
(_, maxLoc, resizing_ratio) = found = template_results[0][1]
(startX, startY) = (int(maxLoc[0] * resizing_ratio), int(maxLoc[1] * resizing_ratio))
(endX, endY) = (int((maxLoc[0] + tW) * resizing_ratio), int((maxLoc[1] + tH) * resizing_ratio))


Calc the hits for template found by using the maxVal on line 36